In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt

In [ ]:
# Load the data
data = pd.read_csv('component_month_counts.csv')
data['Month'] = pd.to_datetime(data['Month'])

# Define the target end date for all components
END_DATE = pd.to_datetime("2024-11-01")

# Define the range of months for each component individually
components = data['Component'].unique()
expanded_data = pd.DataFrame()
last_observed_date = []

for component in components:
    # Filter data for the current component
    component_data = data[data['Component'] == component]

    last_observed_date.append(component_data['Month'].max())
    
    # Define the full range of months for this component
    all_months = pd.date_range(start=component_data['Month'].min(), end=END_DATE, freq='MS')
    
    # Create a DataFrame with all months for this component
    component_months = pd.DataFrame({'Component': component, 'Month': all_months})
    
    # Merge to include counts, filling missing months with zero counts
    component_months = component_months.merge(component_data, on=['Component', 'Month'], how='left').fillna(0)
    component_months['Count'] = component_months['Count'].astype(int)
    
    # Append to the main expanded data
    expanded_data = pd.concat([expanded_data, component_months], ignore_index=True)

In [ ]:
from tqdm import tqdm

traces = []

for component in tqdm(components):
  # Select a specific component for modeling (e.g., "Windows SMB")
  component_data = expanded_data[expanded_data['Component'] == component]
  component_counts = component_data['Count'].values

  # Calculate the time in months since each observation (relative to the earliest date)
  time_since_observation = (component_data['Month'] - component_data['Month'].min()).dt.days // 30

  # PyMC Model
  with pm.Model() as model:
      # Priors for alpha and beta using Exponential distributions
      alpha = pm.Exponential("alpha", 1.0)
      beta = pm.Exponential("beta", 1.0)
      
      # Prior for decay_rate, treating it as an unknown parameter if desired
      decay_rate = pm.Exponential("decay_rate", 1.0)
      
      # Time-decaying lambda using alpha, beta, and learned decay_rate
      lambda_ = pm.Gamma("lambda", alpha, beta)
      lambda_t = lambda_ * np.exp(-decay_rate * time_since_observation)
      
      # Likelihood for observed counts
      counts_observed = pm.Poisson("counts_observed", mu=lambda_t, observed=component_counts)
      
      # Run the NUTS sampler
      trace = pm.sample(3000, tune=1000, target_accept=0.9, return_inferencedata=True)

      traces.append(trace)
      
  # # Plot trace to check convergence
  # pm.plot_trace(trace)
  # plt.show()

  # # Summary of the posterior
  # summary = pm.summary(trace)
  # print(summary)

In [ ]:
import numpy as np

# Define the number of prediction samples and the future time point
num_prediction_samples = 1000
PREDICTION_DATE = pd.to_datetime("2024-12-01") # Predicting 1 month into the future
results = []

for i in tqdm(range(len(traces))):
  future_time = (PREDICTION_DATE - last_observed_date[i]).days // 30

  # Extract posterior samples for each parameter
  decay_rate_samples = traces[i].posterior["decay_rate"].values.flatten()
  alpha_samples = traces[i].posterior["alpha"].values.flatten()
  beta_samples = traces[i].posterior["beta"].values.flatten()
  lambda_samples = traces[i].posterior["lambda"].values.flatten()

  # Generate predictions for the next month
  predicted_counts = []

  for n in range(num_prediction_samples):
      # Draw samples for decay_rate and lambda from the posterior
      decay_rate = decay_rate_samples[n]
      lambda_ = lambda_samples[n]
      
      # Calculate the decayed rate for the future time point
      lambda_future = lambda_ * np.exp(-decay_rate * future_time)
      
      # Draw a predicted count from the Poisson distribution with the decayed rate
      predicted_count = np.random.poisson(lambda_future)
      predicted_counts.append(predicted_count)

  # Calculate summary statistics for the predictions
  predicted_mean = np.mean(predicted_counts)
  predicted_std = np.std(predicted_counts)
  # predicted_95_ci = np.percentile(predicted_counts, [2.5, 97.5])

  # print(f"Predicted Mean Count for Next Month: {predicted_mean}")
  # print(f"Predicted Standard Deviation: {predicted_std}")
  # print(f"95% Prediction Interval: {predicted_95_ci}")

  # results.append([component[i], , mean_beta, mean_lambda,
  #                             predicted_counts_mean, predicted_counts_std])
